## 4. Producing a Training Matrix

We want to take the sliding window approaches detailed above detecting genomic loci of interest, and move towards using those regions of interest to predict TMB as accurately as possible.

In [1]:
setwd("/Users/jacobbradley/Documents/CCG/Code")

### Packages

In [2]:
install.packages("tidyverse")
library(tidyverse)
# Data Handling

if (!requireNamespace("BiocManager"))
    install.packages("BiocManager")
BiocManager::install()
library(BiocManager)
# General bioinformatics packages

library(BSgenome)
library(BSgenome.Hsapiens.UCSC.hg19)
# Human genome build GChr37


The downloaded binary packages are in
	/var/folders/g9/9x6m35fn5ydf88vdlm9n1vg80000gn/T//RtmpmUBMZ9/downloaded_packages


── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.1.0     ✔ purrr   0.2.5
✔ tibble  1.4.2     ✔ dplyr   0.7.8
✔ tidyr   0.8.2     ✔ stringr 1.3.1
✔ readr   1.1.1     ✔ forcats 0.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Loading required namespace: BiocManager
Bioconductor version 3.8 (BiocManager 1.30.3), R 3.5.1 (2018-07-02)
Update old packages: 'AnnotationHub', 'backports', 'BH', 'BiocManager',
  'BiocParallel', 'Biostrings', 'biovizBase', 'broom', 'callr', 'circlize',
  'class', 'clusterProfiler', 'codetools', 'cowplot', 'DESeq', 'EDASeq',
  'edgeR', 'ELMER', 'ensembldb', 'foreign', 'gdsfmt', 'GenomicAlignments',
  'gganimate', 'ggpubr', 'git2r', 'grImport', 'Gviz', 'haven', 'htmlTable',
  'httpuv', 'httr', 'IRdisplay', 'IRkernel', 'jsonlite', 'knitr', 'lattice',
  'limma', 'markdown', 'MASS', 'Matrix', 'mclust', 'mgcv', '

In [3]:
source("TMB_Funcs.txt")
# My Stuff!

### Producing a training matrix

In [4]:
genome <- BSgenome.Hsapiens.UCSC.hg19

In [17]:
new_data <- read_tsv("/Volumes/CCG8/processed_data/brca_tcga/data_mutations_extended.txt")
new_data <- abridge(new_data)

Warning message:
“Missing column names filled in: 'X54' [54]”Parsed with column specification:
cols(
  .default = col_character(),
  Entrez_Gene_Id = col_integer(),
  Start_Position = col_integer(),
  End_Position = col_integer(),
  Score = col_integer(),
  t_ref_count = col_integer(),
  t_alt_count = col_integer(),
  Protein_position = col_integer(),
  Hotspot = col_integer(),
  stop_WU = col_integer(),
  X54 = col_double(),
  tumor_vaf = col_double(),
  normal_ref_reads = col_integer(),
  start_WU = col_integer(),
  strand_WU = col_integer(),
  tumors_var_reads = col_integer(),
  normal_vaf = col_double(),
  tumor_ref_reads = col_integer(),
  normal_var_reads = col_integer()
)
See spec(...) for full column specifications.
Warning message in rbind(names(probs), probs_f):
“number of columns of result is not a multiple of vector length (arg 1)”Warning message:
“3282 parsing failures.
row # A tibble: 5 x 5 col     row col     expected  actual file                                         

In [12]:
starting_window <- 1000000; coverage <- 1; zoom <- 10

In [220]:
#tcga_brca_slide <- RLL_slide(maf_file = new_data, starting_window = 1000000, iterations = 4, zoom = 10, coverage = 1)
#write_tsv(path = '/Users/jacobbradley/Documents/CCG/Data/tcga_brca_slide', tcga_brca_slide  %>%  unnest())
tcga_brca_slide <- read_tsv("/Users/jacobbradley/Documents/CCG/Data/tcga_brca_slide", col_types = list(col_guess(), col_guess(), col_character(), col_guess(), col_guess(), col_guess())) %>% 
    group_by(iteration, jump_length) %>% 
    nest() %>% 
    select(iteration, data, jump_length)

Warning message:
“Unnamed `col_types` should have the same length as `col_names`. Using smaller of the two.”Warning message in rbind(names(probs), probs_f):
“number of columns of result is not a multiple of vector length (arg 1)”Warning message:
“12583 parsing failures.
row # A tibble: 5 x 5 col     row col   expected  actual    file                                          expected   <int> <chr> <chr>     <chr>     <chr>                                         actual 1     1 <NA>  6 columns 7 columns '/Users/jacobbradley/Documents/CCG/Data/tcga… file 2     2 <NA>  6 columns 7 columns '/Users/jacobbradley/Documents/CCG/Data/tcga… row 3     3 <NA>  6 columns 7 columns '/Users/jacobbradley/Documents/CCG/Data/tcga… col 4     4 <NA>  6 columns 7 columns '/Users/jacobbradley/Documents/CCG/Data/tcga… expected 5     5 <NA>  6 columns 7 columns '/Users/jacobbradley/Documents/CCG/Data/tcga…
... ................. ... ...............................................................................

In [223]:
tcga_brca_windows <- RLL_windows(tcga_brca_slide, 1000000, 1)

ERROR: Error in UseMethod("mutate_"): no applicable method for 'mutate_' applied to an object of class "c('double', 'numeric')"


In [187]:
tcga_brca_matrices <- RLL_matrices(new_data, tcga_brca_windows, starting_window = 1000000, coverage = 1)

In [206]:
tcga_brca_matrices$matrices[[2]] %>% 
    filter(TMB > 15) %>%
    mutate(., sum = rowSums(select(., -c("Tumor_Sample_Barcode", "TMB")))) #%>% 
   # select(TMB, sum)

Tumor_Sample_Barcode,TMB,Window_1,Window_2,Window_3,Window_4,Window_5,sum
TCGA-A2-A0T5-01,36.03333,0,2,0,0,1,3
TCGA-A8-A09Z-01,33.46667,0,1,0,1,3,5
TCGA-A8-A0A6-01,68.36667,3,4,1,2,0,10
TCGA-AC-A23H-01,115.90000,0,1,0,4,4,9
TCGA-AN-A046-01,145.16667,1,5,2,7,8,23
TCGA-AN-A0AK-01,30.80000,0,0,0,0,3,3
TCGA-AO-A03M-01,18.43333,0,0,1,0,0,1
TCGA-AO-A128-01,22.86667,0,0,0,0,2,2
TCGA-BH-A0B6-01,16.53333,0,1,0,0,0,1
TCGA-BH-A0HF-01,18.60000,0,0,0,0,3,3


In [ ]:
RLL_models <- function(rll_matrices) {
    if (!("matrices" %in% colnames(rll_matrices))) {
        rll_matrices <- RLL_matrices()
    }
    
    return(mutate(rll_matrices, models = map(matrices, getModel)))
}

In [207]:
tcga_brca_models <- RLL_models(tcga_brca_matrices)

In [208]:
tcga_brca_models %>% 
    mutate(glance = map(models, broom::glance)) %>% 
    select(iteration, glance) %>% 
    unnest()

iteration,r.squared,adj.r.squared,sigma,statistic,p.value,df,logLik,AIC,BIC,deviance,df.residual
1,0.7401116,0.7398462,3.551863,2788.0018,1.022833e-288,2,-2634.368,5274.736,5289.402,12350.800,979
2,0.7801937,0.7790665,3.273195,692.1447,9.845893e-318,6,-2552.207,5118.413,5152.633,10445.961,975
3,0.8788974,0.8773961,2.438334,585.4352,0.000000e+00,13,-2259.815,4547.630,4616.070,5755.217,968
4,0.9398050,0.9362947,1.757636,267.7285,0.000000e+00,55,-1916.932,3945.863,4219.624,2860.676,926


### Validate Model Against a New Dataset

Here's our new dataset:

In [80]:
brca_test <- read_tsv("/Volumes/CCG8/processed_data/coad_tcga_pan_can_atlas_2018/data_mutations_extended.txt")

Parsed with column specification:
cols(
  .default = col_character(),
  Entrez_Gene_Id = col_integer(),
  Start_Position = col_integer(),
  End_Position = col_integer(),
  t_ref_count = col_integer(),
  t_alt_count = col_integer(),
  n_ref_count = col_integer(),
  n_alt_count = col_integer(),
  Protein_position = col_integer(),
  Hotspot = col_integer(),
  NCALLERS = col_integer(),
  n_depth = col_integer(),
  t_depth = col_integer()
)
See spec(...) for full column specifications.


In [14]:
library("biomaRt")

In [15]:
listMarts()

biomart,version
ENSEMBL_MART_ENSEMBL,Ensembl Genes 94
ENSEMBL_MART_MOUSE,Mouse strains 94
ENSEMBL_MART_SNP,Ensembl Variation 94
ENSEMBL_MART_FUNCGEN,Ensembl Regulation 94


In [22]:
ensembl = useMart("ensembl", dataset = "hsapiens_gene_ensembl")

In [52]:
getBM(attributes = c('ensembl_gene_id', 'external_gene_name'),
      filters = c('chromosome_name', 'start', 'end'),
      values = list(2, 27000001,28000000),
      mart = ensembl) %>% 
    nrow()


[1] 55